In [241]:
import stanza
import pickle
import unicodedata
import re
from collections import defaultdict, Counter
stanza.download('ru')
nlp = stanza.Pipeline("ru")

2025-05-27 19:47:07 INFO: Downloaded file to /Users/anwasty/stanza_resources/resources.json
2025-05-27 19:47:07 INFO: Downloading default packages for language: ru (Russian) ...
2025-05-27 19:47:10 INFO: File exists: /Users/anwasty/stanza_resources/ru/default.zip
2025-05-27 19:47:14 INFO: Finished downloading models and saved to /Users/anwasty/stanza_resources
2025-05-27 19:47:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-27 19:47:14 INFO: Downloaded file to /Users/anwasty/stanza_resources/resources.json
2025-05-27 19:47:17 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2025-05-27 19:47:17 INFO: Using device: cpu
2025-05-27 19:47:17 INFO: Loading: tokenize
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the defaul

In [242]:
def load():
    with open(file="lemmas.dat", mode='rb') as f:
        lemmas = pickle.loads(f.read())
    with open(file="wordforms.dat", mode='rb') as f:
        wordforms = pickle.loads(f.read())
    return lemmas, wordforms



def special_cases(text, wordforms):
    """
    Обрабатывает слова с дефисами
    """
    replacements = {}

    words = text.split()
    for word in words:
        key = word.lower()
        if ("-" in word or " " in word) and key in wordforms and len(wordforms[key]) == 1:
            accented = wordforms[key][0]["accentuated"]
            simple_form = key.replace("-", "").replace(" ", "")
            simple_form_cap = simple_form[0].upper() + simple_form[1:]

            replacements[simple_form] = accented
            replacements[simple_form_cap] = accented[0].upper() + accented[1:]

            if word[0].isupper():
                text = text.replace(word, simple_form_cap)
            else:
                text = text.replace(word, simple_form)

    return text, replacements


def get_historic(res, word):
    """
    В устаревших словах проводит замену "нь" на "ни"
    """
    if word.get("historical") and word.get("original"):
        replace_index = res.rfind("ни")
        if (
            replace_index != -1 and
            len(res) - replace_index <= 4 and
            "нь" in word["original"]
        ):
            return res[:replace_index] + "нь" + res[replace_index + 2:]
    return res



def compatible(interpretation, lemma, pos, morph, lemmas):
    """
    Проверяет, соответствуют ли морфологические признаки, определённые моделью
    признакам из словаря wordforms для нахождения единственной ударной формы
    """
    if interpretation == "canonical":
        return True

    form_parts = interpretation.lower().replace("-", " ").replace("/", " ").split()
    morph_feats = set(morph.split("|"))

    cases_number = [
        (["plural"], "Number=Plur"),
        (["singular"], "Number=Sing"),
        (["nominative"], "Case=Nom"),
        (["genitive"], "Case=Gen"),
        (["dative"], "Case=Dat"),
        (["instrumental"], "Case=Ins"),
        (["prepositional", "locative"], "Case=Loc"),  
    ]

    for interp_keys, morph_tag in cases_number:
        if any(k in form_parts for k in interp_keys):
            continue
        if morph_tag in morph_feats:
            return False

    if "Case=Acc" in morph_feats and "accusative" not in form_parts:
        if not (pos == "ADJ" and "Animacy=Inan" in morph_feats):
            return False

    tense = [
        (["present", "future"], "Tense=Past"),
        (["past", "future"], "Tense=Pres"),
        (["past", "present"], "Tense=Fut"),
    ]

    for interp_keys, morph_tag in tense:
        if any(k in form_parts for k in interp_keys) and morph_tag in morph_feats:
            return False

    return True


def single_accentuation(interpretations):
    if len(interpretations) == 0:
        return None
    res = interpretations[0]["accentuated"]
    for i in range(1, len(interpretations)):
        if interpretations[i]["accentuated"] != res:
            return None
    return res


def accentuate_word(word, lemmas):
    """
    Ставит ударение в слове на основе морфлогических признаков и лемме
    """
    if word["is_punctuation"] or ("interpretations" not in word):
        return word["token"]

    else:
        res = single_accentuation(word["interpretations"])
        if not (res is None):
            res = get_historic(res, word)
            return res
        else:
            compatible_interpretations = []
            for i in range(len(word["interpretations"])):
                if compatible(word["interpretations"][i]["form"], word["interpretations"][i]["lemma"], word["pos"], word["morph"], lemmas):
                    compatible_interpretations.append(word["interpretations"][i])
            res = single_accentuation(compatible_interpretations)
            if not (res is None):
                res = get_historic(res, word)
                return res
            else:
                new_compatible_interpretations = []
                for i in range(len(compatible_interpretations)):
                    if compatible_interpretations[i]["lemma"] == word["lemma"]:
                        new_compatible_interpretations.append(compatible_interpretations[i])
                res = single_accentuation(new_compatible_interpretations)
                if not (res is None):
                    res = get_historic(res, word)
                    return res
                else:
                    return word["token"]


def tokenize(text, wordforms):
    """
    Токенизирует строку и извлекает морфологические признаки с помощью Станзы
    """
    res = []
    text, replacements = special_cases(text, wordforms)
    doc = nlp(text)

    for sentence in doc.sentences:
        for token in sentence.words:
            token_text = token.text

            word = {
                "token": token_text,
                "start_char": token.start_char,
                "end_char": token.end_char,
                "line_text": text,
            }

            if token.upos == "PUNCT":
                word.update({
                    "is_punctuation": True,
                    "whitespace": " "
                })
            else:
                original_token = replacements.get(token_text, token_text)
                word.update({
                    "token": original_token,
                    "morph": token.feats or "",
                    "pos": token.upos,
                    "lemma": token.lemma,
                    "is_punctuation": False,
                    "uppercase": original_token.isupper(),
                    "capital_letter": original_token[0].isupper(),
                    "whitespace": " "
                })
                
                if original_token in wordforms:
                    word["interpretations"] = wordforms[original_token]
                elif original_token.lower() in wordforms:
                    word["interpretations"] = wordforms[original_token.lower()]
                else:
                    alt = token_text.replace("нь", "ни")
                    if alt in wordforms:
                        word["interpretations"] = wordforms[alt]
                        word["token"] = alt
                        word["historical"] = True
                        word["original"] = original_token
                    elif alt.lower() in wordforms:
                        word["interpretations"] = wordforms[alt.lower()]
                        word["token"] = alt
                        word["historical"] = True
                        word["original"] = original_token

            res.append(word)

    return res, replacements



tokenized_lines = []  

def accentuate(text, wordforms, lemmas):
    """
    Основная функция, разбивает текст на строки, расставляет ударения
    и возвращает текст с ударениями
    """
    lines = text.splitlines(keepends=True)
    output = []
    global tokenized_lines
    tokenized_lines = [] 

    for line in lines:
        if line.strip() == "":
            output.append(line)
            tokenized_lines.append([]) 
            continue

        words, replacements = tokenize(line, wordforms)
        tokenized_lines.append(words)

        result_line = ""
        for i, word in enumerate(words):
            accented = accentuate_word(word, lemmas)

            if word["token"] in replacements:
                accented = replacements[word["token"]]

            if word.get("capital_letter"):
                accented = accented.capitalize()
            if word.get("uppercase"):
                accented = accented.upper()

            if word["is_punctuation"]:
                result_line = result_line.rstrip() + accented
            else:
                result_line += accented

            if i + 1 < len(words) and not words[i + 1]["is_punctuation"]:
                result_line += " "

        result_line = result_line.rstrip()
        if line.endswith("\n"):
            result_line += "\n"
        output.append(result_line)

    result = "".join(output)
    return result



lemmas, wordforms = load()


In [243]:
vowels = "аеёиоуыэюяАЕЁИОУЫЭЮЯ"

def count_vowels(word):
    return sum(1 for ch in word.lower() if ch in vowels)

def is_mono(word):
    return count_vowels(word) == 1

def is_stress(word):
    return "́" in word or 'ё' in word.lower()

def is_pos(pos, feats):
    if pos in {"NOUN", "ADJ", "ADV", "PROPN", "PRON"}:
        return True
    if pos == "VERB" and "Aux=Yes" not in feats:
        return True
    return False

def stress_to_mono(text, tokenized_lines):
    """
    Расставляет ударения в односложных словах
    """
    lines = text.splitlines(keepends=True)
    new_lines = []

    for line, tokens in zip(lines, tokenized_lines):
        if line.strip() == "":
            new_lines.append(line)
            continue

        mod_line = line

        for i, token in enumerate(tokens):
            if token.get("is_punctuation"):
                continue

            word = token["token"]
            if is_stress(word):
                continue

            stripped = re.sub(r'[^\wёЁа-яА-Я-]', '', word)
            if not is_mono(stripped):
                continue

            pos = token.get("pos", "")
            feats = token.get("morph", "")

            if is_pos(pos, feats) or (
                i + 1 < len(tokens) and tokens[i + 1].get("is_punctuation")
            ):
                for j, ch in enumerate(word):
                    if ch.lower() in vowels:
                        stressed_word = word[:j+1] + "́" + word[j+1:]
                        mod_line = mod_line.replace(word, stressed_word, 1)
                        break

        new_lines.append(mod_line)

    return "".join(new_lines)


In [244]:
def detect_rhythm(accented_text):
    """
    Определяет ритмическую схему строк, где 1 - ударный, 0 - безударный
    """
    lines = accented_text.splitlines() 
    result = []

    for line in lines:
        pattern = []
        i = 0
        while i < len(line):
            letter = line[i]

            if letter.lower() in vowels:
                if letter.lower() == 'ё':
                    is_stress = True
                elif i + 1 < len(line) and line[i + 1] in ["́"]:
                    is_stress = True
                    i += 1
                else:
                    is_stress = False

                pattern.append(1 if is_stress else 0)

            i += 1

        result.append(pattern)

    return result




In [245]:
def detect_meter(patterns):
    """
    Анализирует ритм строки, определяет метр и размер стихотворения
    """
    meters = ["ямб", "хорей", "дактиль", "амфибрахий", "анапест"]
    meter_syllable = {
        "ямб": 2,
        "хорей": 2,
        "дактиль": 3,
        "амфибрахий": 3,
        "анапест": 3
    }

    results = []
    meter_count = {m: 0 for m in meters}
    meter_score = {m: 0.0 for m in meters}
    counted_lines = 0
    meter_for_line = []

    for line_number, pattern in enumerate(patterns):
        if not pattern:
            continue  

        n = len(pattern)
        scores = {}

        iamb_score = sum(
            1 for i in range(n)
            if (i % 2 == 1 and pattern[i] == 1) or (i % 2 == 0 and pattern[i] == 0)
        )
        scores["ямб"] = round(iamb_score / n * 100, 1)

        chorea_score = sum(
            1 for i in range(n)
            if (i % 2 == 0 and pattern[i] == 1) or (i % 2 == 1 and pattern[i] == 0)
        )
        scores["хорей"] = round(chorea_score / n * 100, 1)

        dactyl_score = 0
        feet_count = 0
        for i in range(0, n - 2, 3):
            if pattern[i:i+3] == [1, 0, 0]:
                dactyl_score += 1
            feet_count += 1
        scores["дактиль"] = round(dactyl_score / max(1, feet_count) * 100, 1)

        amphibrach_score = 0
        feet_count = 0
        for i in range(0, n - 2, 3):
            if pattern[i:i+3] == [0, 1, 0]:
                amphibrach_score += 1
            feet_count += 1
        scores["амфибрахий"] = round(amphibrach_score / max(1, feet_count) * 100, 1)

        anapest_score = 0
        feet_count = 0
        for i in range(0, n - 2, 3):
            if pattern[i:i+3] == [0, 0, 1]:
                anapest_score += 1
            feet_count += 1
        scores["анапест"] = round(anapest_score / max(1, feet_count) * 100, 1)

        best_meter = max(scores, key=scores.get)
        meter_for_line.append((pattern, best_meter))

        meter_count[best_meter] += 1
        for m in meters:
            meter_score[m] += scores[m]
        counted_lines += 1

        results.append({
            "строка": line_number + 1,
            "размер": best_meter,
            "оценки": scores
        })

    if counted_lines == 0:
        result_all = {
            "размер": "не определён",
            "строки по метру": {},
            "по среднему": {},
            "лучший по среднему": None
        }
    else:
        average_scores = {
            m: round(meter_score[m] / counted_lines, 1)
            for m in meters
        }
        most_voted = max(meter_count, key=meter_count.get)
        best_avg = max(average_scores, key=average_scores.get)

        sylls_foot = meter_syllable[best_avg]
        foot_position_dict = defaultdict(list)

        lengths = [len(p) for p, m in meter_for_line if m == most_voted]
        if lengths:
            most_common_length, _ = Counter(lengths).most_common(1)[0]
        else:
            most_common_length = 0

        filtered_patterns = [
            p for p, m in meter_for_line
            if m == most_voted and len(p) == most_common_length
        ]

        for pattern in filtered_patterns:
            feet = [pattern[i:i+sylls_foot] for i in range(0, len(pattern), sylls_foot)]
            for pos, foot in enumerate(feet, start=1):
                foot_position_dict[pos].append(foot)

        valid_positions = [
            pos for pos, foot_list in foot_position_dict.items()
            if any(1 in f for f in foot_list)
        ]

        final_result = f"{len(valid_positions)}-стопный {best_avg}"

        result_all = {
            "размер": final_result,
            "строки по метру": meter_count,
            "по среднему": average_scores,
            "лучший по среднему": best_avg
        }

    return results, result_all


In [246]:
soft_vowels = {'е', 'ё', 'и', 'ю', 'я'}
hard_vowels = {'а', 'о', 'у', 'э', 'ы'}
all_vowels = soft_vowels.union(hard_vowels)

always_voiced = {'м', 'н', 'л', 'р', 'й', 'в'}
voiced_consonants = {'б', 'в', 'г', 'д', 'ж', 'з'}
voiceless_consonants = {'п', 'ф', 'к', 'т', 'ш', 'с', 'ц', 'ч', 'щ', 'х'}
all_consonants = voiced_consonants.union(voiceless_consonants).union(always_voiced)

vowel_symbols = ['а', 'о', 'у', 'ы', 'э', 'е', 'ё', 'и', 'ю', 'я']
consonant_symbols = [
    'б', 'в', 'г', 'д', 'ж', 'з', 'п', 'ф', 'к', 'т', 'ш', 'с',
    'ц', 'ч', 'щ', 'х', 'м', 'н', 'л', 'р', 'й'
]

j_map = {'ё': 'о', 'е': 'э', 'ю': 'у', 'я': 'а'}

consonant_equivalents = {
    'б': ['п'], 'в': ['ф'], 'г': ['к'],
    'д': ['т'], 'ж': ['ш'], 'з': ['с'],
    'ц': ['т', 'с'], 'щ': ['ш']
}

soft_equivalents = {
    ('и', 'ы'), ('и', 'е'), ('и', 'я'), ('и', 'а'),
    ('е', 'э'), ('ё', 'о')
}

vowel_ids = {symbol: line_number for line_number, symbol in enumerate(vowel_symbols)}
consonant_ids = {symbol: line_number for line_number, symbol in enumerate(consonant_symbols)}

def symbol_id(symbol):
    if symbol in vowel_ids:
        return vowel_ids[symbol]
    elif symbol in consonant_ids:
        return consonant_ids[symbol]
    return -1


In [247]:
class Phoneme:
    def __init__(self, kind, symbol, accent="none", voiced=None, palatalized=None):
        self.kind = kind 
        self.symbol = symbol
        self.id = symbol_id(symbol)
        self.accent = accent
        self.voiced = voiced
        self.palatalized = palatalized

    def __repr__(self):
        if self.kind == 'vowel':
            return f"V({self.symbol}, id={self.id}, {self.accent})"
        return f"C({self.symbol}, id={self.id}, voiced={self.voiced}, pal={self.palatalized})"
        

In [248]:
def full_phonetic_transcript(word: str):
    """
    Преобразует слово в список фонем
    """
    result = []
    letters = list(word)
    i = 0

    while i < len(letters):
        letter = letters[i].lower()
        accent = "none"

        if i + 1 < len(letters) and letters[i + 1] == '́':
            accent = 'main'
            i += 1

        if letter == 'ё':
            result.append(Phoneme("vowel", 'ё', accent='main'))

        elif letter in vowel_ids:
            result.append(Phoneme("vowel", letter, accent))

        elif letter in consonant_ids:
            voiced = letter in voiced_consonants or letter in always_voiced
            palatalized = (
                i + 1 < len(letters) and (letters[i + 1] == 'ь' or letters[i + 1] in soft_vowels)
            )
            result.append(Phoneme("consonant", letter, voiced=voiced, palatalized=palatalized))

        i += 1

    return result

def normalize_vowel(letter):
    return j_map.get(letter, letter)

def phoneme_distance(p1, p2):
    """
    Оценивает расстояние междум двумя фонемами
    """
    if p1.kind != p2.kind:
        return 1.0

    if p1.kind == "vowel":
        letter1 = normalize_vowel(p1.symbol)
        letter2 = normalize_vowel(p2.symbol)
        if letter1 == letter2:
            return 0.0
        if (letter1, letter2) in soft_equivalents or (letter2, letter1) in soft_equivalents:
            return 0.3 
        return 0.7

    if p1.kind == "consonant":
        letter1 = p1.symbol
        letter2 = p2.symbol
        if letter1 == letter2:
            return 0.0
        if letter2 in consonant_equivalents.get(letter1, []) or letter1 in consonant_equivalents.get(letter2, []):
            return 0.2
        return 1.0

    return 1.0

def extract_rhyme_tail(phonemes):
    """
    Определяет ударный хвост слова: от ударной гласной до конца слова
    """
    for i in range(len(phonemes) - 1, -1, -1):
        p = phonemes[i]
        if p.kind == "vowel" and p.accent == "main":
            return phonemes[i:]
    return []  

def phoneme_sequence_distance(tail1, tail2, allow_truncate=True):
    """
    Вычисляет среднее расстояние между рифмующимися хвостами
    """
    if not tail1 or not tail2:
        return 1.0

    if not allow_truncate and len(tail1) != len(tail2):
        return 1.0

    min_len = min(len(tail1), len(tail2))
    total = 0.0

    for i in range(1, min_len + 1):
        p1 = tail1[-i]
        p2 = tail2[-i]
        total += phoneme_distance(p1, p2)

    distance = total / min_len

    length_cost = abs(len(tail1) - len(tail2)) * 0.1
    return round(distance + length_cost, 3)

def check_rhyme(word1, word2, threshold=0.4):
    p1 = full_phonetic_transcript(word1)
    p2 = full_phonetic_transcript(word2)

    tail1 = extract_rhyme_tail(p1)
    tail2 = extract_rhyme_tail(p2)

    if not tail1 or not tail2:
        return False

    distance = phoneme_sequence_distance(tail1, tail2)
    return distance <= threshold

def get_rhyme_type(phonemes):
    count = 0
    for p in phonemes:
        if p.kind == "vowel":
            count += 1

    if count == 0:
        return "нет ударения"
    elif count == 1:
        return "мужская"
    elif count == 2:
        return "женская"
    elif count == 3:
        return "дактилическая"
    else:
        return "гипердактилическая"

def detect_rhyme_pattern(scheme):
    if len(scheme) != 4 or '-' in scheme:
        return "неопределённая"

    a, b, c, d = scheme

    def same(x, y): return x == y

    if same(a, b) and same(c, d):
        return "парная"
    elif same(a, c) and same(b, d):
        return "перекрёстная"
    elif same(a, d) and same(b, c):
        return "кольцевая"
    elif all(x == a for x in scheme):
        return "сквозная"
    else:
        return "смешанная"

def build_rhyme_scheme(lines, threshold=0.3):
    """
    Определяет рифмовую схему
    """
    endings = []
    tails = []
    valid_values = []

    for line_number, line in enumerate(lines):
        if not line.strip():
            continue
        last_word = line.strip().split()[-1]
        phonemes = full_phonetic_transcript(last_word)
        tail = extract_rhyme_tail(phonemes)
        if not tail:
            endings.append(None)
            tails.append(None)
            continue
        endings.append(last_word)
        tails.append(tail)
        valid_values.append(line_number)

    labels = {}
    rhyme_labels = []
    rhyme_types = []
    next_label = 1

    for i, tail1 in enumerate(tails):
        if tail1 is None:
            rhyme_labels.append(0) 
            rhyme_types.append('—')
            continue

        rhyme_found = False
        for label, index in labels.items():
            for j in index:
                if check_rhyme(endings[i], endings[j], threshold=threshold):
                    rhyme_labels.append(label)
                    rhyme_types.append(get_rhyme_type(tail1))
                    labels[label].append(i)
                    rhyme_found = True
                    break
            if rhyme_found:
                break

        if not rhyme_found:
            labels[next_label] = [i]
            rhyme_labels.append(next_label)
            rhyme_types.append(get_rhyme_type(tail1))
            next_label += 1

    return rhyme_labels, rhyme_types

def describe_rhyme_scheme(scheme):
    """
    Определяет тип рифмовки
    """
    if not scheme or len(scheme) < 2:
        return "нет рифмы"

    normalized = [str(s) for s in scheme]
    n = len(normalized)
    description = set()
    index_used = set()

    i = 0
    while i <= n - 4:
        a, b, c, d = normalized[i:i+4]
        if a == d and b == c and a != b:
            description.add("кольцевая")
            index_used.update({i, i+1, i+2, i+3})
            i += 4
        else:
            i += 1

    for i in range(n - 1):
        if i in index_used or i + 1 in index_used:
            continue
        if normalized[i] == normalized[i + 1]:
            description.add("смежная")
            index_used.update({i, i+1})

    for i in range(n - 3):
        if i in index_used and i+2 in index_used:
            continue

        is_cross = False
        if normalized[i] == normalized[i + 2]:
            is_cross = True
            index_used.update({i, i+2})
        if normalized[i + 1] == normalized[i + 3]:
            is_cross = True
            index_used.update({i+1, i+3})

        if is_cross:
            description.add("перекрёстная")

    if len(set(normalized)) == 1:
        return "сквозная"

    remaining = set(range(n)) - index_used
    if description and remaining:
        return "смешанная: " + ", ".join(description)
    if not description:
        return "смешанная"
    if len(description) == 1:
        return next(iter(description))
    return "смешанная: " + ", ".join(description)

def analyze_rhyme_structure(text_lines):
    if isinstance(text_lines, str):
        text_lines = text_lines.splitlines()

    results = []
    scheme_counter = Counter()

    current_strofa = []
    id_strofa = 1

    for line in text_lines + [""]: 
        if line.strip() == "":
            if current_strofa:
                scheme, rhyme_types = build_rhyme_scheme(current_strofa)
                scheme_type = describe_rhyme_scheme(scheme)

                results.append({
                    "строфа": id_strofa,
                    "схема": scheme,
                    "тип рифмовки": scheme_type,
                    "рифмы по строкам": rhyme_types,
                    "строки": current_strofa
                })
                scheme_counter[scheme_type] += 1
                id_strofa += 1
                current_strofa = []
        else:
            current_strofa.append(line)

    result_rhyming = {
        "рифмовка стихотворения": scheme_counter.most_common(1)[0][0] if scheme_counter else "нет рифмы",
        "все рифмовки": dict(scheme_counter)
    }

    return results, result_rhyming


Ниже можно вставить своё стихотворение. Можете взять одно из выборки стихотворений: https://docs.google.com/spreadsheets/d/1_wLmEnxiYITVwixTEY0I3V24WuQT9yQm3Yb79WcpJM0/edit?usp=sharing

In [258]:
#Вставьте своё стихотворение
poem = """
По вечерам над ресторанами
Горячий воздух дик и глух,
И правит окриками пьяными
Весенний и тлетворный дух.

Вдали над пылью переулочной,
Над скукой загородных дач,
Чуть золотится крендель булочной,
И раздаётся детский плач.

И каждый вечер, за шлагбаумами,
Заламывая котелки,
Среди канав гуляют с дамами
Испытанные остряки.

Над озером скрипят уключины
И раздается женский визг,
А в небе, ко всему приученный
Бессмысленно кривится диск.

И каждый вечер друг единственный
В моем стакане отражён
И влагой терпкой и таинственной,
Как я, смирён и оглушён.

А рядом у соседних столиков
Лакеи сонные торчат,
И пьяницы с глазами кроликов
«In vino veritas!»* кричат.

И каждый вечер, в час назначенный
(Иль это только снится мне?),
Девичий стан, шелками схваченный,
В туманном движется окне.

И медленно, пройдя меж пьяными,
Всегда без спутников, одна
Дыша духами и туманами,
Она садится у окна.

И веют древними поверьями
Ее упругие шелка,
И шляпа с траурными перьями,
И в кольцах узкая рука.

И странной близостью закованный,
Смотрю за темную вуаль,
И вижу берег очарованный
И очарованную даль.

Глухие тайны мне поручены,
Мне чьё-то солнце вручено,
И все души моей излучины
Пронзило терпкое вино.

И перья страуса склонённые
В моем качаются мозгу,
И очи синие бездонные
Цветут на дальнем берегу.

В моей душе лежит сокровище,
И ключ поручен только мне!
Ты право, пьяное чудовище!
Я знаю: истина в вине.
"""

In [259]:
# Расстановка ударений
res = accentuate(poem, wordforms, lemmas)
res_fixed = stress_to_mono(res, tokenized_lines)
print(res_fixed)


По вечера́м над рестора́нами
Горя́чий во́здух ди́к и глу́х,
И пра́вит о́криками пья́ными
Весе́нний и тлетво́рный ду́х.

Вдали́ над пы́лью переу́лочной,
Над ску́кой за́городных да́ч,
Чу́ть золоти́тся кре́ндель бу́лочной,
И раздаётся де́тский пла́ч.

И ка́ждый ве́чер, за шлагба́умами,
Зала́мывая котелки́,
Среди́ кана́в гуля́ют с да́мами
Испы́танные остряки́.

Над о́зером скрипя́т уклю́чины
И раздаётся же́нский ви́зг,
А в не́бе, ко всему́ приу́ченный
Бессмы́сленно криви́тся ди́ск.

И ка́ждый ве́чер дру́г еди́нственный
В моём стака́не отражён
И вла́гой терпкой и таи́нственной,
Как я́, смирён и оглушён.

А ря́дом у сосе́дних сто́ликов
Лаке́и со́нные торча́т,
И пья́ницы с глаза́ми кро́ликов
« In vino veritas!»* крича́т.

И ка́ждый ве́чер, в ча́с назна́ченный
( И́ль э́то то́лько сни́тся мне́?),
Де́вичий ста́н, шелка́ми схва́ченный,
В тума́нном дви́жется окне́.

И ме́дленно, пройдя́ меж пья́ными,
Всегда́ без спу́тников, одна́
Дыша́ ду́хами и тума́нами,
Она́ сади́тся у окна́.

И ве́ют дре́вним

In [260]:
# Ритмическая схема стихотворения
schemes = detect_rhythm(res_fixed)
for i in schemes:
    print(i)

[]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 1, 0, 1]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 1]
[]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 0, 1, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1]
[]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1]
[]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 1]
[]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 1]
[]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 0]
[1, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 0, 1, 0, 1, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1]
[]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 1]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
[0, 1

In [261]:
# Размер стихотворения
results, result_all = detect_meter(schemes)
print(result_all["размер"])

4-стопный ямб


In [262]:
# Количество строк, соответствующих метру стихотворения
for meter, i in result_all["строки по метру"].items():
    print(f"{meter}: {i} строк")

ямб: 51 строк
хорей: 0 строк
дактиль: 0 строк
амфибрахий: 0 строк
анапест: 1 строк


In [263]:
# Анализ по каждой строке
for i in results:
    print(f"Строка {i['строка']}: размер = {i['размер']}, оценки = {i['оценки']}")

Строка 2: размер = ямб, оценки = {'ямб': 70.0, 'хорей': 30.0, 'дактиль': 33.3, 'амфибрахий': 33.3, 'анапест': 0.0}
Строка 3: размер = ямб, оценки = {'ямб': 100.0, 'хорей': 0.0, 'дактиль': 0.0, 'амфибрахий': 50.0, 'анапест': 0.0}
Строка 4: размер = ямб, оценки = {'ямб': 80.0, 'хорей': 20.0, 'дактиль': 33.3, 'амфибрахий': 66.7, 'анапест': 0.0}
Строка 5: размер = ямб, оценки = {'ямб': 87.5, 'хорей': 12.5, 'дактиль': 0.0, 'амфибрахий': 50.0, 'анапест': 50.0}
Строка 7: размер = ямб, оценки = {'ямб': 80.0, 'хорей': 20.0, 'дактиль': 33.3, 'амфибрахий': 66.7, 'анапест': 0.0}
Строка 8: размер = ямб, оценки = {'ямб': 87.5, 'хорей': 12.5, 'дактиль': 50.0, 'амфибрахий': 50.0, 'анапест': 0.0}
Строка 9: размер = ямб, оценки = {'ямб': 70.0, 'хорей': 30.0, 'дактиль': 33.3, 'амфибрахий': 33.3, 'анапест': 0.0}
Строка 10: размер = ямб, оценки = {'ямб': 87.5, 'хорей': 12.5, 'дактиль': 0.0, 'амфибрахий': 0.0, 'анапест': 0.0}
Строка 12: размер = ямб, оценки = {'ямб': 81.8, 'хорей': 18.2, 'дактиль': 33.3, 'а

In [264]:
# Рифмовка стихотворения
results, result_rhyming = analyze_rhyme_structure(res_fixed)
print(result_rhyming["рифмовка стихотворения"])

перекрёстная


In [265]:
# Рифмовка по каждой строфе
for i in results:
    print(f"Строфа {i['строфа']}: {i['тип рифмовки']} (схема: {' '.join(map(str, i['схема']))})")

Строфа 1: перекрёстная (схема: 1 2 1 2)
Строфа 2: перекрёстная (схема: 1 2 1 2)
Строфа 3: перекрёстная (схема: 1 2 1 2)
Строфа 4: смешанная: перекрёстная (схема: 1 2 3 2)
Строфа 5: перекрёстная (схема: 1 2 1 2)
Строфа 6: перекрёстная (схема: 1 2 1 2)
Строфа 7: перекрёстная (схема: 1 2 1 2)
Строфа 8: перекрёстная (схема: 1 2 1 2)
Строфа 9: перекрёстная (схема: 1 2 1 2)
Строфа 10: перекрёстная (схема: 1 2 1 2)
Строфа 11: перекрёстная (схема: 1 2 1 2)
Строфа 12: перекрёстная (схема: 1 2 1 2)
Строфа 13: смешанная: перекрёстная (схема: 1 2 1 0)


In [266]:
# Тип рифмы для строк
rhymes_ev = []
rhymes_od = []

for i in results:
    for i, rhyme_type in enumerate(i["рифмы по строкам"]):
        if rhyme_type != '—':
            if i % 2 == 0:
                rhymes_od.append(rhyme_type)
            else:
                rhymes_ev.append(rhyme_type)
                
for type, rhymes in [("Чётные", rhymes_ev), ("Нечётные", rhymes_od)]:
    result = Counter(rhymes).most_common(1)
    if top:
        print(f"{type} строки: {result[0][0]} рифма")

Чётные строки: мужская рифма
Нечётные строки: дактилическая рифма
